<a href="https://colab.research.google.com/github/CharlyWheels/Carlos_Rueda_TFM/blob/colab/Labepc_LSTM_padding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [69]:
import json
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [70]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/labdata.csv', engine='python')
df.shape

(1173, 38)

In [71]:
print(df.columns)

Index(['id', 'reference', 'epc', 'score', 'count', 'maxRssiValue',
       'minRssiValue', 'maxTimeStamp', 'minTimeStamp', 'rssiDiff', 'seenDiff',
       'antenna1', 'antenna2', 'antenna3', 'antenna4', 'ts_percent0',
       'ts_percent1', 'ts_percent2', 'ts_percent3', 'ts_percent4',
       'ts_percent5', 'ts_percent6', 'ts_percent7', 'ts_percent8',
       'rv_percent0', 'rv_percent1', 'rv_percent2', 'rv_percent3',
       'rv_percent4', 'rv_percent5', 'rv_percent6', 'rv_percent7',
       'rv_percent8', 'lastModification', 'timeStamp', 'rssiValue', 'antennas',
       'desired'],
      dtype='object')


In [72]:
df.drop(['id', 'reference','epc','score', 'count', 'maxRssiValue',
        'minRssiValue', 'maxTimeStamp', 'minTimeStamp', 'rssiDiff',
        'seenDiff', 'antenna1', 'antenna2', 'antenna3', 'antenna4', 'ts_percent0',
       'ts_percent1', 'ts_percent2', 'ts_percent3', 'ts_percent4',
       'ts_percent5', 'ts_percent6', 'ts_percent7', 'ts_percent8',
       'rv_percent0', 'rv_percent1', 'rv_percent2', 'rv_percent3',
       'rv_percent4', 'rv_percent5', 'rv_percent6', 'rv_percent7',
       'rv_percent8', 'lastModification'], axis=1, inplace=True)
df.shape

(1173, 4)

In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1173 entries, 0 to 1172
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timeStamp  1173 non-null   object
 1   rssiValue  1173 non-null   object
 2   antennas   1173 non-null   object
 3   desired    1173 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 36.8+ KB


In [74]:
from sklearn.utils import resample

df_majority = df[df.desired==1]
df_minority = df[df.desired==0]

df_minority_upsampled = resample(df_minority, replace=True, n_samples=1088, random_state=123)

In [75]:
df_upsampled = pd.concat([df_majority, df_minority_upsampled]) # type: ignore

In [76]:
X = []
Y = []
for index, row in df_upsampled.iterrows():
  X_ts = np.pad(np.array(json.loads(row['timeStamp'])), (175 - len(np.array(json.loads(row['timeStamp']))), 0), mode='constant')
  X_rv = np.pad(np.array(json.loads(row['rssiValue'])), (175 - len(np.array(json.loads(row['rssiValue']))), 0), mode='constant')
  X_an = np.pad(np.array(json.loads(row['antennas'])), (175 - len(np.array(json.loads(row['antennas']))), 0), mode='constant')
  Y_item = row['desired']
  X_item = np.array(list(zip(X_ts,X_rv,X_an)))
  X.append(X_item)
  Y.append(Y_item)

print(np.shape(X))
print(np.shape(Y))


(2176, 175, 3)
(2176,)


In [77]:
print(type(X))
print(np.shape(X))

<class 'list'>
(2176, 175, 3)


In [88]:
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

x_train, x_test, y_train, y_test=train_test_split(X,Y, test_size=0.10, random_state=42)
y_train=to_categorical(y_train)
y_test=to_categorical(y_test)

In [98]:
from keras.layers import Embedding,Dense,LSTM,Dropout,Flatten,BatchNormalization,Conv1D,GlobalMaxPooling1D,MaxPooling1D
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

model = Sequential()
model.add(LSTM(12, input_shape=(175, 3), dropout=0.7, recurrent_dropout=0.7))
model.add(Dense(6))
model.add(Dropout(0.9))
model.add(Dense(2, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.01), metrics=['accuracy'])

In [99]:
es = EarlyStopping(monitor='val_loss', mode='min',patience=5)

In [100]:
print(np.shape(x_train))
print(np.shape(y_train))

(1958, 175, 3)
(1958, 2)


In [101]:
history_lstm = model.fit(x_train, y_train, epochs=120,batch_size=1,validation_data=(x_test,y_test),shuffle=False)

ValueError: ignored